# 1D Maze Place Cell Analysis

Workflow for analyzing place cells in a 3D maze with tube corridors.
Tube traversals are serialized onto a concatenated 1D axis and analyzed
using the same metrics (SI, stability, place fields) as the 2D pipeline.

In [ ]:
from pathlib import Path

import matplotlib.pyplot as plt
from IPython.display import display
from tqdm.auto import tqdm

from placecell.dataset import BasePlaceCellDataset
from placecell.notebook import (
    browse_units_1d,
    create_deconv_browser,
    create_shuffle_browser_1d,
)
from placecell.visualization import (
    plot_diagnostics,
    plot_graph_overlay,
    plot_occupancy_preview_1d,
    plot_shuffle_test_1d,
    plot_summary_scatter,
)

## Configuration

Point to a maze analysis config and a per-session data paths YAML.

In [ ]:
CONFIG_ID = "pcell_maze_config"
DATA_PATH = Path(
    "/Volumes/ProcData/minizero_analysis/202512round/"
    "202512_analysis_3dmaze/20251219/WL25/WL25_20251219.yaml"
)

ds = BasePlaceCellDataset.from_yaml(CONFIG_ID, DATA_PATH)
print(type(ds).__name__)  # Should print 'MazeDataset'

## Step 1: Load & Preprocess

In [ ]:
ds.load()

In [ ]:
ds.preprocess_behavior()

In [ ]:
if ds.graph_polylines is not None:
    plot_graph_overlay(
        ds.graph_polylines,
        ds.graph_mm_per_pixel,
        tube_order=ds.maze_cfg.tube_order,
        video_frame=ds.behavior_video_frame,
    )
    plt.show()
else:
    print("No behavior graph — skipping graph overlay.")

In [ ]:
ds.deconvolve(progress_bar=tqdm)

In [ ]:
%matplotlib widget

fig_d, controls_d = create_deconv_browser(
    ds.good_unit_ids, ds.traces, ds.S_list,
    ds.neural_fps, ds.cfg.neural.trace_name,
    time_window=ds.spatial_1d.trace_time_window,
)
plt.show()
display(controls_d)

## Step 2: Match Events & Occupancy

In [ ]:
ds.match_events()
ds.compute_occupancy()

In [ ]:
plot_occupancy_preview_1d(
    ds.trajectory_1d_filtered,
    ds.occupancy_time,
    ds.valid_mask,
    ds.edges_1d,
    trajectory_1d=ds.trajectory_1d,
    tube_boundaries=ds.tube_boundaries,
    tube_labels=ds.effective_tube_order,
);

## Step 3: Analyze Units

In [ ]:
ds.analyze_units(progress_bar=tqdm)

In [ ]:
summary = ds.summary()
print(summary)

In [ ]:
plot_summary_scatter(ds.unit_results, p_value_threshold=ds.spatial_1d.p_value_threshold);

In [ ]:
plot_diagnostics(ds.unit_results, p_value_threshold=ds.spatial_1d.p_value_threshold);

## Population rate map & per-unit shuffle tests

**Population heatmap** — rate maps of all place cells sorted by peak position (static).

**Per-unit browser** — use the slider to view each unit's rate map, SI shuffle distribution,
and stability shuffle distribution individually.

In [ ]:
plot_shuffle_test_1d(
    ds.unit_results,
    ds.edges_1d,
    p_value_threshold=ds.spatial_1d.p_value_threshold,
    tube_boundaries=ds.tube_boundaries,
    tube_labels=ds.effective_tube_order,
);

In [ ]:
%matplotlib widget

fig_shuf, controls_shuf = create_shuffle_browser_1d(
    ds.unit_results,
    ds.edges_1d,
    p_value_threshold=ds.spatial_1d.p_value_threshold,
    tube_boundaries=ds.tube_boundaries,
    tube_labels=ds.effective_tube_order,
)
plt.show()
display(controls_shuf)

## Interactive Unit Browser

Browse individual units: rate maps (1st half / 2nd half / full) and calcium trace with events.

In [ ]:
%matplotlib widget

fig_units, controls_units = browse_units_1d(ds)
plt.show()
display(controls_units)

## Save Bundle

In [ ]:
ds.save_bundle(f"../user_data/bundles/{DATA_PATH.stem}_sandbox.pcellbundle")